In [1]:
# TODO support layers, units
import ray
import dni
import importlib
importlib.reload(dni)
import numpy as np
ray.shutdown()
ray.init(address="10.128.0.51:6379")

2019-11-05 23:56:17,504	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.


{'node_ip_address': '10.128.0.51',
 'object_store_address': '/tmp/ray/session_2019-11-05_16-45-18_491016_1575/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-05_16-45-18_491016_1575/sockets/raylet',
 'redis_address': '10.128.0.51:6379',
 'session_dir': '/tmp/ray/session_2019-11-05_16-45-18_491016_1575',
 'webui_url': None}

In [2]:
@ray.remote
class Scanner(dni.access.Scan):
    def __init__(self):
        from google.cloud import storage
        import numpy as np
        storage_client = storage.Client()
        bucketn = "dni-storage"
        bucket = storage_client.get_bucket(bucketn)
        self.blob = bucket.get_blob('Datasets/data.txt')
        self.curr = 0
        self.step = 128 # fetch how many data at one time
    def Next(self):
        if not self.HasMore():
            return ""
        # fetch data
        s = self.blob.download_as_string(start=self.curr*51*self.step, 
                                         end=self.curr*51*self.step+51*self.step-1)
        self.curr += 1
        return self.process_raw(s)

    def HasMore(self):
        return self.curr * self.step < 4992
    
    def process_raw(self,s):
        s = s.decode("utf-8") 
        s = s.split("\n")
        if s[-1] == '':
            del s[-1]
        char2int = {'X': 0, '0':1, '1':2, '|':3, '&':4}
        encoded_seqs = [[char2int[x] for x in seq] for seq in s]
        n_chars = max(char2int.values()) + 1
        seq_len = len(encoded_seqs[0])
        X = np.zeros((len(encoded_seqs), seq_len, n_chars), dtype=np.int)
        for i, enc_seq in enumerate(encoded_seqs):
            for j, x in enumerate(enc_seq):
                X[i, j, x] = 1
        return X
    
@ray.remote
class Act_extract(dni.model.Extractor):
    def __init__(self, modelname, layer):
        from google.cloud import storage
        storage_client = storage.Client()
        bucket = storage_client.get_bucket("dni-storage")
        blob = bucket.get_blob(modelname)
        import os.path
        if not os.path.isfile(modelname.split("/")[-1]):
            blob.download_to_filename(modelname.split("/")[-1])
        from keras.models import Model, load_model
        self.model = load_model(modelname.split("/")[-1])
        self.model._make_predict_function()
        outputs = [self.model.layers[l].output for l in layer]
        self.model = Model(inputs = self.model.input, outputs = outputs)
    def extract(self,input,unit):
        pred = self.model.predict(input)
        if not isinstance(pred, list):
            pred = [pred]
        for i in range(len(pred)):
            pred[i] = pred[i][...,unit]
        return pred
    
# feature builder    
def fsm_states(transition_table, dictionary, init_state = 0, id_fsm = None):
    def F(seq_raw):
        int2char = {0:'X', 1:'0', 2:'1', 3:'|', 4:'&'}
        seq = ''
        #latter not 0
        for i in seq_raw:
            for k in range(len(i)):
                if i[k] == 1:
                    seq = seq + int2char[k]
        features = np.zeros(len(seq))
        cur_state = init_state
        for i, x in enumerate(seq):
            cur_state = transition_table[cur_state][dictionary[x]]
            features[i] = cur_state
        return features
    return F

fsm_dict = {'X': 0, '0':1, '1':2, '|':3, '&':4}
fsm_tbl =  [[0, 2, 1, -1, -1],
            [0, -1, -1, 4, 3],
            [0, -1, -1, 6, 5],
            [0, 2, 1, -1, -1],
            [0, 1, 1, -1, -1],
            [0, 2, 2, -1, -1],
            [0, 2, 1, -1, -1]]
fsm_states_7 = fsm_states(fsm_tbl, fsm_dict, id_fsm = '7')

fsm_tbl4 = [[0,2,1,-1,-1],
            [0,1,1,1,3],
            [0,2,2,3,2],
            [0,2,1,2,1]]
fsm_states_4 = fsm_states(fsm_tbl4, fsm_dict, id_fsm = '4')


fsm_tbl3 = [[0,2,1,-1,-1],
           [0,2,1,1,0],
           [0,2,2,0,2]]
fsm_states_3 = fsm_states(fsm_tbl3, fsm_dict, id_fsm = '3')


features = [fsm_states_7, fsm_states_4, fsm_states_3]

def random_fsm(n_states, fsm_dict=fsm_dict):
    s = np.random.randint(n_states, size=n_states*len(fsm_dict))
    tran = np.reshape(s, (n_states, len(fsm_dict)))
    return tran

fsm_states_3rd = fsm_states(random_fsm(3), fsm_dict, id_fsm = 'rd_3')
fsm_states_4rd = fsm_states(random_fsm(4), fsm_dict, id_fsm = 'rd_4')
fsm_states_7rd = fsm_states(random_fsm(7), fsm_dict, id_fsm = 'rd_7')

features += [fsm_states_3rd, fsm_states_4rd, fsm_states_7rd]
feature_names = ["states_7", "states_4", "states_3","states_3rd","states_4rd","states_7rd"]

@ray.remote  
class F_Extractor(dni.feature.Batch_Feature_Extractor):
    pass
    
@ray.remote 
class corr_metric(dni.metric.Batch_Exhaustive_Correlation):
    pass

Metric_names = ["Corr"]

In [3]:
import dni
import importlib
importlib.reload(dni)
# system decides whether cache, batch
# feature factory, batching, caching
# do it for the sql
# for feature factory, feature

#
dni.inspect(clusterid = "10.128.0.51:6379",
            AccessMethod= Scanner,
            ActivationExt = Act_extract, Neuron = [[[0],[0,1,2,3]]], 
            Models = ["Models/boolean.h5"],
            FeaturesFunctions = features, FeatureNames = feature_names, 
            FeatureExtractor = [F_Extractor]*6,
            MetricExtractor = [corr_metric], MetricName = Metric_names)

2019-11-05 23:56:29,493	WARNING worker.py:1268 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.


(pid=7973) Using TensorFlow backend.
(pid=7973) WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(pid=7973) Instructions for updating:
(pid=7973) If using Keras pass *_constraint arguments to layers.
(pid=7973) 2019-11-05 23:56:34.091886: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
(pid=7973) To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
(pid=7973) 2019-11-05 23:56:34.099726: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
(pid=7973) 2019-11-05 23:56:34.100091: I tensorflow/compiler/xla/service/service.cc:168] XLA servic